<center><img src='https://1.bp.blogspot.com/-WWe6LvkQZLY/UfWViMco9wI/AAAAAAAAAN8/dzibibpS7Xo/w1200-h630-p-k-no-nu/titanic_in_the_night_wallpaper-401299-1283323224.jpeg' width = '800px'></center>

<center><img src='https://i.imgur.com/SZQXq1q.png' width = '600px'></center>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

<h1 id="heading1">
<span style="font-size:35px;color:#20beff;font-weight:bold"><center>Understanding Data</center></span>
      <a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading1" target ="_self"></a>
</h1>

In [ ]:
df = pd.read_csv('../input/titanic/train.csv')

In [ ]:
df

<span style='font-size:25px;font-weight:bold;color:#20beff'>Correlation</span>

In [ ]:
#check if there is any significant correlation
df.corr()

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(df.corr(),annot = True,cmap='Oranges_r',square = False,yticklabels = False,linewidth = 2)

In [ ]:
df.Survived.value_counts()

In [ ]:
plt.figure(figsize=(10,6),facecolor = '#90eef8')
plt.pie(df.Survived.value_counts(),labels=['Not Survived','Survived'],startangle=90,explode=[0,0.05],pctdistance=0.8,shadow=True,colors=['#fdb98a','#f76502'],autopct = '%1.1f%%')
centre_circle = plt.Circle((0,0),0.60,fc='#90eef8')
plt.gca().add_artist(centre_circle)
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
df[df.Survived == 0].Sex.value_counts(),df[df.Survived == 1].Sex.value_counts()

In [ ]:
_,ax = plt.subplots(1,2,figsize=(20,6),facecolor = '#90eef8')
ax[0].pie(df[df.Survived == 0].Sex.value_counts(),labels=['Male','Female'],startangle=90,explode=[0,0.05],pctdistance=0.8,shadow=True,colors=['#fdb98a','#f76502'],autopct = '%1.1f%%')
ax[1].pie(df[df.Survived == 1].Sex.value_counts(),labels=['Female','Male'],startangle=90,explode=[0,0.05],pctdistance=0.8,shadow=True,colors=['#fdb98a','#f76502'],autopct = '%1.1f%%')

centre_circle = plt.Circle((0,0),0.60,fc='#90eef8')
ax[0].add_artist(centre_circle)
centre_circle = plt.Circle((0,0),0.60,fc='#90eef8')
ax[1].add_artist(centre_circle)


ax[0].set_title('Not Survived',fontsize = 30)
ax[1].set_title('Survived',fontsize = 30)
plt.tight_layout()
plt.show()

<span style="font-size:20px;font-weight:bold;color:#20beff">Probability of surviving for females are higher than males.</span>

<span style='font-size:25px;font-weight:bold;color:#20beff'>NaN</span>

In [ ]:
df.isna().sum()

In [ ]:
df.Embarked.describe()

In [ ]:
#Filling NaN value with most frequent value
df.Embarked.fillna('S',inplace=True)

In [ ]:
le = LabelEncoder()
df.Cabin = le.fit_transform(df.Cabin)

In [ ]:
sns.kdeplot(df.Cabin)

In [ ]:
df.Cabin = df.Cabin.replace(147,np.nan)

In [ ]:
#As Cabin variable is not symetrically distributed and right skewed. So, we will use median over mean
df['Cabin'].fillna(df.groupby('Pclass')['Cabin'].transform('median'),inplace = True)

In [ ]:
sns.kdeplot(df.Age)

In [ ]:
df['Age'].fillna(df.groupby('Pclass')['Age'].transform('median'),inplace = True)

<span style='font-size:25px;font-weight:bold;color:#20beff'>Categorising age in different sub groups.</span>
 
*   0 < Age < 18 -->  0
*  18 < Age < 30 -->  1
*  30 < Age < 50 -->  2
*  50 < Age < 80 -->  3
*   0 < Age      -->  4 

In [ ]:
for i in range(0,df.shape[0]):
    if df.iloc[i,5] < 18 :
        df.iloc[i,5] = 0
    elif 18 <= df.iloc[i,5] < 30 :
        df.iloc[i,5] = 1
    elif 30 <= df.iloc[i,5] < 50 :
        df.iloc[i,5] = 2
    elif 50 <= df.iloc[i,5] < 80 :
        df.iloc[i,5] = 3
    if df.iloc[i,5] >= 80 :
        df.iloc[i,5] = 4

<span style='font-size:25px;font-weight:bold;color:#20beff'>Both SibSp and Parch combined is representing the number of family member</span>

In [ ]:
df['Family'] = df['SibSp']+df['Parch']

In [ ]:
df.iloc[:,3].map(lambda x : x.split(',')[1].split('.')[0].strip()).value_counts()

In [ ]:
df.iloc[:,3] = df.iloc[:,3].map(lambda x : x.split(',')[1].split('.')[0].strip())

In [ ]:
for i in range (0,df.shape[0]):
    if df.iloc[i,3] == 'Mr' or df.iloc[i,3] == 'Mrs' or df.iloc[i,3] == 0:
        df.iloc[i,3] = 0
    elif df.iloc[i,3] == 'Master' or df.iloc[i,3] == 'Miss' or df.iloc[i,3] == 1:
        df.iloc[i,3] = 1
    else:
        df.iloc[i,3] = 2
    

In [ ]:
df

In [ ]:
x = df.drop(['PassengerId','Survived','Ticket','SibSp','Parch','Cabin'],axis =1)

In [ ]:
x = pd.get_dummies(x)

In [ ]:
y = df.Survived

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=6)


<h1 id="heading2">
<span style="font-size:35px;color:#20beff;font-weight:bold"><center>Model Fitting</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading2" target ='_self'></a>
</h1>

<span style='font-size:25px;font-weight:bold;color:#20beff'>Random Forest</span>

In [ ]:
rfc = RandomForestClassifier(random_state=3)
rfc.fit(x_train,y_train)
accuracy_score(y_test,rfc.predict(x_test))

<span style='font-size:25px;font-weight:bold;color:#20beff'>LightGBM</span>

In [ ]:
lgb = LGBMClassifier(learning_rate=0.1,min_child_samples=14,reg_alpha=7,reg_lambda=7)
lgb.fit(x_train,y_train)
accuracy_score(y_test,lgb.predict(x_test))

<span style='font-size:25px;font-weight:bold;color:#20beff'>XGBoost</span>

In [ ]:
xgb = XGBClassifier(learning_rate=0.9,reg_alpha=0.1,reg_lambda=13)
xgb.fit(x_train,y_train)
accuracy_score(y_test,xgb.predict(x_test))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

<span style='font-size:25px;font-weight:bold;color:#20beff'>K-Fold on LightGBM </span>

In [ ]:
def cval(x, y,estimator,params,folds):

    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=17)
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(x, y)):
        x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
        x_test, y_test = x.iloc[test_idx], y.iloc[test_idx]

        model = estimator(**params)
        model.fit(x_train, y_train, eval_set=[(x_test, y_test)], early_stopping_rounds=100, verbose=0)
        accu = accuracy_score(y_test, model.predict(x_test))
        
        print(str(fold+1)+' Fold Accuracy: '+str(accu))
        print("-"*50)
    
    return model

In [ ]:
params = {'learning_rate':0.1,'min_child_samples':14,'reg_alpha':7,'reg_lambda':7}

In [ ]:
lgbm = cval(x,y,LGBMClassifier,params,5)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=6)

In [ ]:
accuracy_score(y_test,lgbm.predict(x_test))

<h1 id="heading3">
<span style="font-size:35px;color:#20beff;font-weight:bold"><center>Test data</center></span>
<a class="anchor-link" href="https://www.kaggle.com/alampalsingh/notebook004d896e8c#heading3" target ='_self'></a>
</h1>

In [ ]:
test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
test.isna().sum()

In [ ]:
test.Cabin = le.fit_transform(test.Cabin)

In [ ]:
df.Cabin = df.Cabin.replace(76,np.nan)

In [ ]:
test.Cabin.fillna(test.groupby('Pclass')['Cabin'].transform('median'),inplace=True)

In [ ]:
test.Age.fillna(test.groupby('Pclass')['Age'].transform('median'),inplace=True)

In [ ]:
test.Fare.describe()

In [ ]:
a = test.Fare.median()

In [ ]:
test.Fare.fillna(a,inplace=True)

In [ ]:
test['Family'] = test['SibSp']+test['Parch']

In [ ]:
test = test.drop(['Cabin','Ticket','SibSp','Parch'],axis =1)

In [ ]:
test

In [ ]:
for i in range(0,test.shape[0]):
    if test.iloc[i,4] < 18 :
        test.iloc[i,4] = 0
    elif 18 <= test.iloc[i,4] < 30 :
        test.iloc[i,4] = 1
    elif 30 <= test.iloc[i,4] < 50 :
        test.iloc[i,4] = 2
    elif 50 <= test.iloc[i,4] < 80 :
        test.iloc[i,4] = 3
    if test.iloc[i,4] >= 80 :
        test.iloc[i,4] = 4

In [ ]:
test

In [ ]:
b = test.PassengerId


In [ ]:
test.Name = test.Name.map(lambda x : x.split(',')[1].split('.')[0].strip())

In [ ]:
for i in range (0,test.shape[0]):
    if test.iloc[i,2] == 'Mr' or test.iloc[i,2] == 'Mrs' or test.iloc[i,2] == 0:
        test.iloc[i,2] = 0
    elif test.iloc[i,2] == 'Master' or test.iloc[i,2] == 'Miss' or test.iloc[i,2] == 1:
        test.iloc[i,2] = 1
    else:
        test.iloc[i,2] = 2

In [ ]:
test = pd.get_dummies(test)

In [ ]:
test.drop(['PassengerId'],axis = 1,inplace= True)

In [ ]:
pred = lgb.predict(test)

In [ ]:
submission = pd.DataFrame({'PassengerId':b,'Survived':pred})

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)